# Every output value are calculated from the preprocessed text

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import glob
from pathlib import Path
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import string
import re

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

# Stopword

In [2]:
stop_words = []
p = Path(r"C:\Users\SOUMYA\@DS&AI\stop")
path_of_the_directory = p
print("Files and directories in a specified path:")
file = Path(path_of_the_directory ).glob('*')
for i in file:
    print(i)
    file = open(i, 'rt')
    text = file.read()
    file.close()
    # split into words by white space
    words = text.split()
    #print(words)
    print(len(words))
    for word in words:
        stop_words.append(word)
len(stop_words)

Files and directories in a specified path:
C:\Users\SOUMYA\@DS&AI\stop\StopWords_Auditor.txt
8
C:\Users\SOUMYA\@DS&AI\stop\StopWords_Currencies.txt
280
C:\Users\SOUMYA\@DS&AI\stop\StopWords_DatesandNumbers.txt
121
C:\Users\SOUMYA\@DS&AI\stop\StopWords_Generic.txt
121
C:\Users\SOUMYA\@DS&AI\stop\StopWords_GenericLong.txt
571
C:\Users\SOUMYA\@DS&AI\stop\StopWords_Geographic.txt
207
C:\Users\SOUMYA\@DS&AI\stop\StopWords_Names.txt
13028


14336

# Positive and Negative dictionary

In [3]:
df=pd.read_csv("Loughran-McDonald_MasterDictionary_1993-2021.csv",nrows=100000)
df.head()

,Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Syllables,Source
0,AARDVARK,1,354,1.549791e-08,1.422414e-08,3.815378e-06,99,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.313382e-10,8.653817e-12,9.241714e-09,1,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,9,3.940147e-10,1.169670e-10,5.290404e-08,7,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,29,1.269603e-09,6.654728e-10,1.595099e-07,28,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,8570,3.751896e-07,3.809452e-07,3.529355e-05,1108,0,0,0,0,0,0,0,3,12of12inf


In [4]:
positive = []
negative = []
for i in range(0, df.shape[0]):
    if df['Word'][i] not in stop_words:
        if df['Negative'][i] != 0:
            negative.append(df['Word'][i])
        elif df['Positive'][i] != 0:
            positive.append(df['Word'][i])

In [5]:
for i in range(len(positive)):
    positive[i] = positive[i].lower()
for i in range(len(negative)):
    negative[i] = negative[i].lower()

# Data

In [6]:
input_data = pd.read_csv('input.csv')
input_data.shape

(170, 2)

In [8]:
# ---------------------------------------------
total_stopwords = set(stopwords.words('english'))
HTMLTAGS = re.compile('<.*?>')
table = str.maketrans(dict.fromkeys(string.punctuation))
remove_digits = str.maketrans('', '', string.digits)
MULTIPLE_WHITESPACE = re.compile(r"\s+")
# ---------------------------------------------
def preprocessor(review):
    # remove html tags
    review = HTMLTAGS.sub(r'', review)

    # remove puncutuation
    review = review.translate(table)
    
    # remove digits
    review = review.translate(remove_digits)
    
    # lower case all letters
    review = review.lower()
    
    # replace multiple white spaces with single space
    review = MULTIPLE_WHITESPACE.sub(" ", review).strip()
    
    # remove stop words
    review = [word for word in review.split()
              if word not in total_stopwords]
    
    # stemming
    #review = ' '.join([stemmer.stem(word) for word in review])
    review = ' '.join([word for word in review])
    
    return review

In [9]:
import spacy
from textstat.textstat import textstatistics,legacy_round

def break_sentences(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    return list(doc.sents)

In [10]:
def word_count(sentences_data):
    words = 0
    for sentence in sentences_data:
        words += len([token for token in sentence])
    return words

In [11]:
def sentence_count(text):
    return len(sentences)

In [12]:
def avg_sentence_length(x, y):
    average_sentence_length = float(x / y)
    return average_sentence_length

In [13]:
def syllables_count(word):
    return textstatistics().syllable_count(word)

In [14]:
def avg_syllables_per_word(text, words):
    syllable = syllables_count(text)
    ASPW = float(syllable) / float(words)
    return legacy_round(ASPW, 1)

In [15]:
def complex_words(text):
     
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    # Find all words in the text
    words = []
    sentences = break_sentences(text)
    for sentence in sentences:
        words += [str(token) for token in sentence]
 
    # difficult words are those with syllables >= 2
    # easy_word_set is provide by Textstat as
    # a list of common words
    com_words_set = set()
     
    for word in words:
        syllable_count = syllables_count(word)
        if word not in nlp.Defaults.stop_words and syllable_count > 2:
            com_words_set.add(word)
 
    return len(com_words_set)

In [16]:
def avg_word_len(txt):
    words = txt.split()
    avg = sum(len(word) for word in words) / len(words)
    return avg

In [17]:
def pronoun_count(txt):
    pronounRegex = re.compile(r'(I|we|my|ours|(?-i:us))',re.I)
    pronouns = pronounRegex.findall(txt)
    return len(pronouns)

In [18]:
# static_url = 'https://insights.blackcoffer.com/all-you-need-to-know-about-online-marketing/'
# response = requests.get(static_url,  headers=headers)
# soup = BeautifulSoup(response.text, "html.parser")
# soup.title
# for title in soup.find_all('title'):
#     print(title.get_text())
# t = []
# for txt in soup.find_all('p'):
#     #print(txt.get_text())
#     t.append(txt.get_text())
# raw_txt = ' '.join([a for a in t]) 
# #print(raw_txt)
# #sentence extraxtion
# sentences = break_sentences(raw_txt)
# #print(sentences)

# #preprocessing extracted sentence
# preprocesed_sent = []
# for sentence in sentences:
#     preprocesed_sent.append(preprocessor(sentence.text))
# #print(preprocesed_sent)

# prep = ' '.join([a for a in preprocesed_sent])

# p = n = 0
# l = []
# for i in range(0, len(preprocesed_sent)):
#     l.append(word_count(preprocesed_sent[i]))
#     for j in preprocesed_sent[i].split():
#         if j in positive:
#             p += 1
#         elif j in negative:
#             n += 1
# tot_processed = sum(a for a in l)
# polarity = (p - n)/((p+n) + 0.000001)
# sub = (p + n)/((tot_processed) + 0.000001)
# complex_count = complex_words(prep)
# avg_sent = avg_sentence_length(tot_processed,sentence_count(preprocesed_sent))
# per_complex = complex_count/tot_processed
# Fog_Index = 0.4 * (avg_sent + per_complex)
# avg_word = tot_processed/sentence_count(preprocesed_sent)
# avg_syll = avg_syllables_per_word(prep, tot_processed)
# pronouns_count = pronoun_count(prep)
# print("Positive count: {}".format(p))
# print("Negative count: {}".format(n))
# print("Polarity: {}".format(polarity))
# print("Subjectivity: {}".format(sub))
# print("avg_sentence_length count: {}".format(avg_sent))
# print("Complex: {}".format(per_complex))
# print("Fog: {}".format(Fog_Index))
# print("Avg words: {}".format(avg_word))
# print("Word count: {}".format(tot_processed))
# print("Complex count: {}".format(complex_count))
# print("avg_syll count: {}".format(avg_syll))
# print("pronoun_count: {}".format(pronouns_count))
# print("Avg word: {}".format(avg_word_len(prep)))

In [25]:
#f = open('URL_ID.txt', 'w')
output = []
for i in range(0, len(input_data)):
    out = []
    print(i)
    static_url = input_data['URL'][i]
    out.append(input_data['URL_ID'][i])
    out.append(input_data['URL'][i])
    #print(static_url)
    response = requests.get(static_url,  headers=headers)
    print(response)
    soup = BeautifulSoup(response.text, "html.parser")
    soup.title
    for title in soup.find_all('title'):
        print(title.get_text())  
    t = []
    for txt in soup.find_all('p'):
        #print(txt.get_text())
        t.append(txt.get_text())
    raw_txt = ' '.join([a for a in t])
    sentences = break_sentences(raw_txt)
    preprocesed_sent = []
    for sentence in sentences:
        preprocesed_sent.append(preprocessor(sentence.text))
    #print(preprocesed_sent)
    prep = ' '.join([a for a in preprocesed_sent])
    p = 0
    n = 0
    l = []
    for i in range(0, len(preprocesed_sent)):
        l.append(word_count(preprocesed_sent[i]))
        for j in preprocesed_sent[i].split():
            if j in positive:
                p += 1
            elif j in negative:
                n += 1
    tot_processed = sum(a for a in l)
    polarity = (p - n)/((p+n) + 0.000001)
    sub = (p + n)/((tot_processed) + 0.000001)
    complex_count = complex_words(prep)
    avg_sent = avg_sentence_length(tot_processed,sentence_count(preprocesed_sent))
    per_complex = complex_count/tot_processed
    Fog_Index = 0.4 * (avg_sent + per_complex)
    avg_word = tot_processed/sentence_count(preprocesed_sent)
    avg_syll = avg_syllables_per_word(prep, tot_processed)
    pronouns_count = pronoun_count(prep)
    
    out.append(p)
    out.append(n)
    out.append(polarity)
    out.append(sub)
    out.append(avg_sent)
    out.append(per_complex)
    out.append(Fog_Index)
    out.append(avg_word)
    out.append(complex_count)
    out.append(tot_processed)
    out.append(avg_syll)
    out.append(pronouns_count)
    out.append(avg_word_len(prep))
    print(out)
    output.append(out)
#     print("Positive count: {}".format(p))
#     print("Negative count: {}".format(n))
#     print("Polarity: {}".format(polarity))
#     print("Subjectivity: {}".format(sub))
#     print("avg_sentence_length count: {}".format(avg_sent))
#     print("Complex: {}".format(per_complex))
#     print("Fog: {}".format(Fog_Index))
#     print("Avg words: {}".format(avg_word))
#     print("Word count: {}".format(tot_processed))
#     print("Complex count: {}".format(complex_count))
#     print("avg_syll count: {}".format(avg_syll))
#     print("pronoun_count: {}".format(pronouns_count))
#     print("Avg word: {}".format(avg_word_len(prep)))
        


0
<Response [200]>
How is Login Logout Time Tracking for Employees in Office done by AI? - Blackcoffer Insights
[1, 'https://insights.blackcoffer.com/how-is-login-logout-time-tracking-for-employees-in-office-done-by-ai/', 4, 5, -0.11111109876543349, 0.0029220779211291954, 123.2, 0.025974025974025976, 49.29038961038961, 123.2, 80, 3080, 0.3, 240, 6.591687041564792]
1
<Response [200]>
How does AI help to monitor Retail Shelf watches? - Blackcoffer Insights
[2, 'https://insights.blackcoffer.com/how-does-ai-help-to-monitor-retail-shelf-watches/', 9, 6, 0.19999998666666757, 0.005121201773601502, 108.48148148148148, 0.03277569136223967, 43.40570286913749, 108.48148148148148, 96, 2929, 0.3, 216, 6.882666666666666]
2
<Response [200]>
AI and its impact on the Fashion Industry - Blackcoffer Insights
[3, 'https://insights.blackcoffer.com/ai-and-its-impact-on-the-fashion-industry/', 31, 22, 0.16981131755072987, 0.006334408986932663, 105.9113924050633, 0.029401219074937254, 42.376317449655296, 105.

[24, 'https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/', 44, 25, 0.27536231484982154, 0.010772833721971454, 95.59701492537313, 0.02576112412177986, 38.249110419797965, 95.59701492537313, 165, 6405, 0.3, 472, 6.778846153846154]
24
<Response [200]>
How machine learning will affect your business? - Blackcoffer Insights
[25, 'https://insights.blackcoffer.com/how-machine-learning-will-affect-your-business/', 11, 6, 0.2941176297577865, 0.006019830026196944, 78.44444444444444, 0.02018413597733711, 31.38585143216871, 78.44444444444444, 57, 2824, 0.3, 197, 6.371134020618556]
25
<Response [200]>
Deep learning impact on areas of e-learning? - Blackcoffer Insights
[26, 'https://insights.blackcoffer.com/deep-learning-impact-on-areas-of-e-learning/', 27, 33, -0.09999999833333337, 0.006647462884262412, 103.74712643678161, 0.02415244848216264, 41.50851155410552, 103.74712643678161, 218, 9026, 0.3, 745, 6.682967959527825]
26
<Response [200]>
How to pro

<Response [200]>
Socio-technical considerations - Embedding care robots into society
[49, 'https://insights.blackcoffer.com/embedding-care-robots-into-society-socio-technical-considerations/', 3, 0, 0.9999996666667778, 0.004431314616792741, 75.22222222222223, 0.022156573116691284, 30.097751518135567, 75.22222222222223, 15, 677, 0.3, 50, 6.145833333333333]
48
<Response [200]>
future digitalization of healthcare services | management challenges
[50, 'https://insights.blackcoffer.com/management-challenges-for-future-digitalization-of-healthcare-services/', 13, 12, 0.03999999840000006, 0.007550588943657328, 106.80645161290323, 0.026578073089700997, 42.73321187439717, 106.80645161290323, 88, 3311, 0.3, 283, 7.334164588528679]
49
<Response [200]>
Are we any closer to preventing a nuclear holocaust? - Blackcoffer Insights
[51, 'https://insights.blackcoffer.com/are-we-any-closer-to-preventing-a-nuclear-holocaust/', 0, 7, -0.9999998571428775, 0.013384321198117933, 65.375, 0.022944550669216062, 

<Response [200]>
Human Rights Outlook - Blackcoffer Insights
[72, 'https://insights.blackcoffer.com/human-rights-outlook/', 0, 0, 0.0, 0.0, 71.28571428571429, 0.02404809619238477, 28.523904952762674, 71.28571428571429, 12, 499, 0.3, 33, 6.228571428571429]
71
<Response [200]>
How Voice search makes your business a successful business. - Blackcoffer Insights
[73, 'https://insights.blackcoffer.com/how-voice-search-makes-your-business-a-successful-business/', 14, 2, 0.7499999531250029, 0.0046096225858226385, 91.34210526315789, 0.021031403053874964, 36.5452546664847, 91.34210526315789, 73, 3471, 0.2, 278, 6.481876332622601]
72
<Response [200]>
How the COVID-19 crisis is redefining jobs and services? - Blackcoffer Insights
[74, 'https://insights.blackcoffer.com/how-the-covid-19-crisis-is-redefining-jobs-and-services/', 25, 30, -0.09090908925619838, 0.007947976877464165, 57.666666666666664, 0.02254335260115607, 23.075684007707128, 57.666666666666664, 156, 6920, 0.3, 551, 6.753303964757709]
73

[95, 'https://insights.blackcoffer.com/estimating-the-impact-of-covid-19-on-the-world-of-work/', 22, 50, -0.38888888348765444, 0.011542160947171823, 93.1044776119403, 0.020519397242705996, 37.2499988036732, 93.1044776119403, 128, 6238, 0.3, 461, 6.63317191283293]
94
<Response [200]>
COVID-19: How have countries been responding? - Blackcoffer Insights
[96, 'https://insights.blackcoffer.com/covid-19-how-have-countries-been-responding-2/', 13, 39, -0.49999999038461557, 0.007817197834062357, 107.29032258064517, 0.02570655441972339, 42.92641165402596, 107.29032258064517, 171, 6652, 0.3, 593, 6.7979094076655056]
95
<Response [200]>
How will COVID-19 affect the world of work? - Blackcoffer Insights
[97, 'https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-4/', 5, 22, -0.6296296063100145, 0.009189925116000707, 183.625, 0.015997277059223963, 73.4563989108237, 183.625, 47, 2938, 0.2, 221, 5.549889135254989]
96
<Response [200]>
Lessons from the past: Some key learnings rel

<Response [200]>
What do you think is the lesson (or lessons) to be learned with Covid 19? - Blackcoffer Insights
[118, 'https://insights.blackcoffer.com/what-do-you-think-is-the-lesson-or-lessons-to-be-learned-with-covid-19/', 0, 5, -0.99999980000004, 0.005458515277883717, 114.5, 0.026200873362445413, 45.81048034934498, 114.5, 24, 916, 0.3, 74, 6.392]
117
<Response [200]>
Coronavirus, the unexpected challenge for the European union - Blackcoffer Insights
[119, 'https://insights.blackcoffer.com/coronavirus-the-unexpected-challenge-for-the-european-union/', 3, 11, -0.5714285306122479, 0.013779527545492592, 78.15384615384616, 0.038385826771653545, 31.27689279224713, 78.15384615384616, 39, 1016, 0.3, 82, 6.854961832061068]
118
<Response [200]>
INDUSTRIAL REVOLUTION 4.0 – PROS AND CONS | Blackcoffer insights
[120, 'https://insights.blackcoffer.com/industrial-revolution-4-0-pros-and-cons/', 0, 0, 0.0, 0.0, 90.875, 0.037138927097661624, 36.36485557083906, 90.875, 27, 727, 0.3, 71, 7.06593406

<Response [200]>
How Political Leaders will Shape Tomorrow using Big Data & Analytics - Blackcoffer Insights
[143, 'https://insights.blackcoffer.com/how-political-leaders-will-shape-tomorrow-using-big-data-analytics/', 27, 13, 0.34999999125000025, 0.007441860463731746, 137.82051282051282, 0.024186046511627906, 55.137879546809785, 137.82051282051282, 130, 5375, 0.3, 387, 7.127627627627628]
142
<Response [200]>
Big Data and Analytics to Help form Political Leaders Win Election - Blackcoffer Insights
[144, 'https://insights.blackcoffer.com/big-data-and-analytics-to-help-form-political-leaders-win-election/', 12, 23, -0.3142857053061227, 0.008311564946969469, 77.98148148148148, 0.027546900973640466, 31.20361135298205, 77.98148148148148, 116, 4211, 0.3, 365, 7.0]
143
<Response [200]>
The Emergence of Data Analytics - Blackcoffer Insights
[145, 'https://insights.blackcoffer.com/the-emergence-of-data-analytics/', 2, 5, -0.4285713673469475, 0.002716336824712326, 95.44444444444444, 0.0283275126

<Response [200]>
Data Driven Dashboards, Foundation and Application in Industry - Blackcoffer Insights
[166, 'https://insights.blackcoffer.com/data-driven-dashboards/', 4, 2, 0.33333327777778704, 0.0017236426309325933, 82.88095238095238, 0.018960068945705257, 33.159964979959234, 82.88095238095238, 66, 3481, 0.3, 236, 6.8991031390134525]
165
<Response [200]>
Role of Big Data in Academia - Blackcoffer Insights
[167, 'https://insights.blackcoffer.com/role-big-data-academia/', 16, 38, -0.4074073998628259, 0.008302583024553722, 91.6056338028169, 0.023677736777367772, 36.65172461583771, 91.6056338028169, 154, 6504, 0.3, 481, 6.893157262905162]
166
<Response [200]>
Methods for Sales Forecasting in Retail - Blackcoffer Insights
[168, 'https://insights.blackcoffer.com/sales-forecasting-in-retail/', 18, 11, 0.24137930202140337, 0.008111888109619053, 127.67857142857143, 0.02909090909090909, 51.08306493506494, 127.67857142857143, 104, 3575, 0.3, 286, 7.340277777777778]
167
<Response [200]>
Detect 

In [26]:
output

[[1,
  'https://insights.blackcoffer.com/how-is-login-logout-time-tracking-for-employees-in-office-done-by-ai/',
  4,
  5,
  -0.11111109876543349,
  0.0029220779211291954,
  123.2,
  0.025974025974025976,
  49.29038961038961,
  123.2,
  80,
  3080,
  0.3,
  240,
  6.591687041564792],
 [2,
  'https://insights.blackcoffer.com/how-does-ai-help-to-monitor-retail-shelf-watches/',
  9,
  6,
  0.19999998666666757,
  0.005121201773601502,
  108.48148148148148,
  0.03277569136223967,
  43.40570286913749,
  108.48148148148148,
  96,
  2929,
  0.3,
  216,
  6.882666666666666],
 [3,
  'https://insights.blackcoffer.com/ai-and-its-impact-on-the-fashion-industry/',
  31,
  22,
  0.16981131755072987,
  0.006334408986932663,
  105.9113924050633,
  0.029401219074937254,
  42.376317449655296,
  105.9113924050633,
  246,
  8367,
  0.3,
  767,
  7.074569789674952],
 [4,
  'https://insights.blackcoffer.com/how-do-deep-learning-models-predict-old-and-new-drugs-that-are-successfully-treated-in-healthcare/',
 

In [27]:
output_data = pd.DataFrame(output, columns=['URL_ID','URL','POSITIVE SCORE','NEGATIVE SCORE',
                                            'POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH',
                                            'PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE',
                                            'COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH'])

In [28]:
output_data

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,1,https://insights.blackcoffer.com/how-is-login-...,4,5,-0.111111,0.002922,123.200000,0.025974,49.290390,123.200000,80,3080,0.3,240,6.591687
1,2,https://insights.blackcoffer.com/how-does-ai-h...,9,6,0.200000,0.005121,108.481481,0.032776,43.405703,108.481481,96,2929,0.3,216,6.882667
2,3,https://insights.blackcoffer.com/ai-and-its-im...,31,22,0.169811,0.006334,105.911392,0.029401,42.376317,105.911392,246,8367,0.3,767,7.074570
3,4,https://insights.blackcoffer.com/how-do-deep-l...,4,1,0.600000,0.002550,140.071429,0.033146,56.041830,140.071429,65,1961,0.3,172,7.061224
4,5,https://insights.blackcoffer.com/how-artificia...,11,13,-0.083333,0.007947,77.435897,0.030132,30.986412,77.435897,91,3020,0.3,289,6.741772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,167,https://insights.blackcoffer.com/role-big-data...,16,38,-0.407407,0.008303,91.605634,0.023678,36.651725,91.605634,154,6504,0.3,481,6.893157
166,168,https://insights.blackcoffer.com/sales-forecas...,18,11,0.241379,0.008112,127.678571,0.029091,51.083065,127.678571,104,3575,0.3,286,7.340278
167,169,https://insights.blackcoffer.com/detect-data-e...,4,50,-0.851852,0.011477,94.100000,0.025717,37.650287,94.100000,121,4705,0.3,418,7.253472
168,170,https://insights.blackcoffer.com/data-exfiltra...,4,4,0.000000,0.004094,93.047619,0.028147,37.230307,93.047619,55,1954,0.3,145,6.655039


In [29]:
output_data.to_csv('output_data.csv')